In [98]:
import pandas as pd

melb_data = pd.read_csv("data/melb_data_ps.csv")
melb_df = melb_data.copy()

# Работа с датами в DataFrame

## ПРИЗНАКИ ДАТЫ И ВРЕМЕНИ

Например, в наших данных об объектах недвижимости есть признак даты продажи — столбец Date. Из данного признака можно выделить массу полезной информации, например год, месяц или день недели продажи имущества. На рынке недвижимости, как известно, присутствует сезонность: есть периоды, когда недвижимость покупается чаще, а есть интервалы времени, когда рынок претерпевает застой, поэтому было бы неплохо учитывать эту сезонность при анализе рынка.

## ФОРМАТ DATETIME

<div class="alert alert-warning">Таким форматом в Pandas является формат datetime, который записывается как YYYY-MM-DD HH: MM: SS, то есть составляющие времени указываются в следующем порядке: год, месяц, день, час, минута, секунда.</div>

В наших данных дата записана в виде DD/MM/YYYY, например 3/12/2017. Посмотрим на это:


In [99]:
display(melb_df["Date"])

0         3/12/2016
1         4/02/2016
2         4/03/2017
3         4/03/2017
4         4/06/2016
            ...    
13575    26/08/2017
13576    26/08/2017
13577    26/08/2017
13578    26/08/2017
13579    26/08/2017
Name: Date, Length: 13580, dtype: object

Для того чтобы преобразовывать столбцы с датами, записанными в распространённых форматах, в формат <code>datetime</code>, можно воспользоваться функцией <code>pandas.to_datetime()</code>. В нашем случае в функции нужно указать параметр <code>dayfirst=True</code>, который будет обозначать, что в первоначальном признаке первым идет день. Преобразуем столбец Date в формат <code>datetime</code>, передав его в эту функцию:


In [100]:
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
display(melb_df['Date'])

0       2016-12-03
1       2016-02-04
2       2017-03-04
3       2017-03-04
4       2016-06-04
           ...    
13575   2017-08-26
13576   2017-08-26
13577   2017-08-26
13578   2017-08-26
13579   2017-08-26
Name: Date, Length: 13580, dtype: datetime64[ns]

## ВЫДЕЛЕНИЕ АТРИБУТОВ DATETIME

<b>Аксессор</b> — это атрибут столбца, хранящий переменные, которые были строковым представлением времени, а затем были изменены с помощью <code>pd.to_datetime()</code>.

Тип данных datetime позволяет с помощью специального аксессора dt выделять составляющие времени из каждого элемента столбца, такие как:

- date — дата;
- year, month, day — год, месяц, день;
- time — время;
- hour, minute, second — час, минута, секунда;
- dayofweek — номер дня недели, от 0 до 6, где 0 — понедельник, 6 — воскресенье;
- day_name — название дня недели;
- dayofyear — порядковый день года;
- quarter — квартал (интервал в три месяца).


In [101]:
years_sold = melb_df["Date"].dt.year
print(years_sold)
print("Min year sold:", years_sold.min())
print("Max year sold:", years_sold.max())
print("Mode year sold:", years_sold.mode()[0])

0        2016
1        2016
2        2017
3        2017
4        2016
         ... 
13575    2017
13576    2017
13577    2017
13578    2017
13579    2017
Name: Date, Length: 13580, dtype: int32
Min year sold: 2016
Max year sold: 2017
Mode year sold: 2017


<p class="alert alert-warning">В результате обращения к атрибуту <code>datetime melb_df['Date'].dt.year</code> мы получаем объект Series, в котором в качестве значений выступают годы продажи объектов недвижимости. Мы можем занести результат в переменную <code>year_sold</code> и далее работать с ней как с обычным столбцом Series — вычислять максимум, минимум и модальное значение.</p>

<p class="alert alert-warning"><b>Примечание.</b> Так как модальных значений в столбце может быть несколько, метод <code>mode()</code> возвращает объект Series, даже если мода в данных только одна. Чтобы сохранить стилистику вывода информации о годе продажи и выводить только число, а не Series, мы обращаемся к результату работы метода <code>mode()</code> по индексу 0.</p>

Теперь попробуем понять, на какие месяцы приходится пик продаж объектов недвижимости. Для этого выделим атрибут dt.month и на этот раз занесём результат в столбец MonthSale, а затем найдём относительную частоту продаж для каждого месяца от общего количества продаж — для этого используем метод value_counts() с параметром normalize (вывод в долях):

In [102]:
melb_df['MonthSale'] = melb_df['Date'].dt.month
melb_df['MonthSale'].value_counts(normalize=True)

MonthSale
5     0.149411
7     0.145950
9     0.135862
6     0.134757
8     0.114138
11    0.082032
4     0.069882
3     0.049926
12    0.044698
10    0.040574
2     0.032622
1     0.000147
Name: proportion, dtype: float64

## РАБОТА С ИНТЕРВАЛАМИ

Часто бывает такая ситуация, что необходимо вычислять интервалы между двумя временными промежутками. Например, можно вычислить, сколько дней прошло с 1 января 2016 года до момента продажи объекта. Для этого можно просто найти разницу между датами продаж и заявленной датой, представленной в формате datetime:

In [103]:
delta_days = melb_df['Date'] - pd.to_datetime('2016-01-01') 
display(delta_days)

0       337 days
1        34 days
2       428 days
3       428 days
4       155 days
          ...   
13575   603 days
13576   603 days
13577   603 days
13578   603 days
13579   603 days
Name: Date, Length: 13580, dtype: timedelta64[ns]

В результате мы получаем Series, элементами которой является количество дней, которое прошло с 1 января 2016 года. Обратите внимание, что данные такого формата относятся к типу timedelta.

Чтобы превратить количество дней из формата интервала в формат целого числа дней, можно воспользоваться аксессором dt для формата timedelta и извлечь из него атрибут days:


In [104]:
display(delta_days.dt.days)

0        337
1         34
2        428
3        428
4        155
        ... 
13575    603
13576    603
13577    603
13578    603
13579    603
Name: Date, Length: 13580, dtype: int64

Рассмотрим другой пример. Давайте создадим признак возраста объекта недвижимости в годах на момент продажи. Для этого выделим из столбца с датой продажи год и вычтем из него год постройки здания. Результат оформим в виде столбца AgeBuilding:


In [105]:
melb_df["AgeBuilding"] = melb_df["Date"].dt.year - melb_df["YearBuilt"]
display(melb_df["AgeBuilding"])

0         46
1        116
2        117
3         47
4          2
        ... 
13575     36
13576     22
13577     20
13578     97
13579     97
Name: AgeBuilding, Length: 13580, dtype: int64

Примечание. Обратите внимание, что, так как года кодируются целым числом, в результате мы тоже получаем целочисленный столбец — тип int64 (а не timedelta).

На самом деле столбец AgeBuilding дублирует информацию столбца YearBuilt, так как, зная год постройки здания, мы автоматически знаем его возраст. Такие признаки не стоит оставлять вместе, поэтому оставим возраст здания, так как он является более наглядным, а год постройки удалим из таблицы:


In [106]:
melb_df = melb_df.drop("YearBuilt", axis=1)

In [107]:
melb_df["WeekdaySale"] = melb_df["Date"].dt.dayofweek

weekend_count = melb_df["WeekdaySale"].isin([5, 6]).sum()
display(weekend_count)

12822